In [7]:
import numpy as np
from models import *
from utils import *
from sampling import *
from dataset import *
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [62]:
x = torch.rand(5, 3)
x

tensor([[0.6064, 0.6833, 0.2786],
        [0.9629, 0.0082, 0.6613],
        [0.5054, 0.2514, 0.9907],
        [0.8925, 0.2343, 0.5892],
        [0.4606, 0.4191, 0.9157]])

In [54]:
x = torch.t(x)

In [55]:
x

tensor([[0.8125, 0.7881, 0.9647, 0.9816, 0.7331],
        [0.8200, 0.5806, 0.5038, 0.3847, 0.6097],
        [0.1538, 0.5388, 0.3565, 0.1733, 0.3223]])

In [56]:
x = x.sort()[0]

In [57]:
x

tensor([[0.7331, 0.7881, 0.8125, 0.9647, 0.9816],
        [0.3847, 0.5038, 0.5806, 0.6097, 0.8200],
        [0.1538, 0.1733, 0.3223, 0.3565, 0.5388]])

In [58]:
x.shape
u = x[:, 1:-1]
u.shape

torch.Size([3, 3])

In [59]:
u = u.mean(dim=-1)
u

tensor([0.8551, 0.5647, 0.2840])

In [72]:
torch.dist(u, x[0])

tensor(0.2756)

In [70]:
dist = torch.cdist(u.unsqueeze(0), x)

In [73]:
d[0]

tensor([0.2756, 0.6809, 0.8484, 0.4514, 0.7588])

In [61]:
sum([0.7881, 0.8125, 0.9647])/3

0.8551000000000001

In [2]:
MODEL = VGG('VGG16')

In [7]:
MODEL.named_parameters

<bound method Module.named_parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, c

In [9]:
model_dict = MODEL.state_dict()

In [24]:
model_dict['features.20.bias'].shape

torch.Size([256])

In [7]:
FRAGMENTS_GROUPS = {
    0:['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 
       'features.1.running_var', 'features.1.num_batches_tracked', 'features.3.weight', 'features.3.bias'],
    
    1:['features.4.weight', 'features.4.bias', 'features.4.running_mean', 'features.4.running_var', 
       'features.4.num_batches_tracked', 'features.7.weight', 'features.7.bias'],
    
    2:['features.8.weight', 'features.8.bias', 'features.8.running_mean', 'features.8.running_var', 
       'features.8.num_batches_tracked', 'features.10.weight', 'features.10.bias', 'features.11.weight', 'features.11.bias', 
       'features.11.running_mean', 'features.11.running_var', 'features.11.num_batches_tracked'],
    
    3:['features.14.weight', 'features.14.bias', 'features.15.weight', 'features.15.bias', 'features.15.running_mean', 
       'features.15.running_var', 'features.15.num_batches_tracked', 'features.17.weight', 'features.17.bias',
       'features.18.weight', 'features.18.bias', 'features.18.running_mean', 'features.18.running_var',
       'features.18.num_batches_tracked']
    , 'features.20.weight', 'features.20.bias', 'features.21.weight', 
       'features.21.bias', 'features.21.running_mean', 'features.21.running_var', 'features.21.num_batches_tracked'],
    
    5:['features.24.weight', 'features.24.bias', 'features.25.weight', 'features.25.bias', 'features.25.running_mean', 
       'features.25.running_var', 'features.25.num_batches_tracked', 'features.27.weight', 'features.27.bias', 
       'features.28.weight', 'features.28.bias', 'features.28.running_mean', 'features.28.running_var', 
       'features.28.num_batches_tracked'],
    
    6:['features.30.weight', 'features.30.bias', 'features.31.weight', 'features.31.bias', 'features.31.running_mean', 
       'features.31.running_var', 'features.31.num_batches_tracked', 'features.34.weight', 'features.34.bias'],
    
    7:['features.35.weight', 'features.35.bias', 'features.35.running_mean', 'features.35.running_var', 
       'features.35.num_batches_tracked', 'features.37.weight', 'features.37.bias', 'features.38.weight', 'features.38.bias', 
       'features.38.running_mean', 'features.38.running_var', 'features.38.num_batches_tracked'],
    
    8:['features.40.weight', 'features.40.bias', 'features.41.weight', 'features.41.bias', 'features.41.running_mean', 
       'features.41.running_var', 'features.41.num_batches_tracked'],
    
    9:['classifier.weight'],
    10:['classifier.bias']
}

In [8]:
FRAGMENTS_GROUPS

{0: ['features.0.weight',
  'features.0.bias',
  'features.1.weight',
  'features.1.bias',
  'features.1.running_mean',
  'features.1.running_var',
  'features.1.num_batches_tracked',
  'features.3.weight',
  'features.3.bias'],
 1: ['features.4.weight',
  'features.4.bias',
  'features.4.running_mean',
  'features.4.running_var',
  'features.4.num_batches_tracked',
  'features.7.weight',
  'features.7.bias'],
 2: ['features.8.weight',
  'features.8.bias',
  'features.8.running_mean',
  'features.8.running_var',
  'features.8.num_batches_tracked',
  'features.10.weight',
  'features.10.bias',
  'features.11.weight',
  'features.11.bias',
  'features.11.running_mean',
  'features.11.running_var',
  'features.11.num_batches_tracked'],
 3: ['features.14.weight',
  'features.14.bias',
  'features.15.weight',
  'features.15.bias',
  'features.15.running_mean',
  'features.15.running_var',
  'features.15.num_batches_tracked',
  'features.17.weight',
  'features.17.bias'],
 4: ['features.18.w

In [3]:
trainset, testset = get_original_cifar10_dataset()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
data = combine_datasets([trainset, testset])

In [ ]:
class9_dataset = DatasetByClass(data, which_class=9)
dataloader =  DataLoader(class9_dataset, batch_size = 1, shuffle=False)

In [ ]:
modelfflnoattack = CNNMNIST().cuda()
modelfflattacknodetection = CNNMNIST().cuda()
modelfflattackdetection = CNNMNIST().cuda()

In [ ]:
dict1 = torch.load('pretrained_models/CNNMNISTMNIST_no_attack_FFL.pth')
dict2 = torch.load('pretrained_models/CNNMNISTMNIST_backdoor_FFL.pth')
dict3 = torch.load('pretrained_models/CNNMNISTMNIST_backdoor_FFLDetectionReputation.pth')

In [ ]:
modelfflnoattack.load_state_dict(dict1)
modelfflattacknodetection.load_state_dict(dict2)
modelfflattackdetection.load_state_dict(dict3)

In [ ]:
def test(model, test_loader):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    test_loss = []
    wrong_predictions = []
    wrong_labels = []
    correct = 0
    for i, (data, target) in enumerate(test_loader):
        data, target = data.cuda(), target.cuda()
        output = model(data)
        test_loss.append(criterion(output, target).item()) # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        if pred != target:
            wrong_predictions.append(i)
            wrong_labels.append(pred)
            
    test_loss = np.mean(test_loss)
    print('\nAverage test loss: {:.4f}, Test accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, 
        len(test_loader),
        100*correct / (len(test_loader))))
    
    return np.round((float(correct) / len(test_loader)), 2), wrong_predictions, wrong_labels

In [ ]:
acc1, wrong_predictions1, wrong_labels1 = test(modelfflnoattack, dataloader)
acc2, wrong_predictions2, wrong_labels2 = test(modelfflattacknodetection, dataloader)
acc3, wrong_predictions3, wrong_label3 = test(modelfflattackdetection, dataloader)

In [ ]:
diff = [ ]
labels = []
i = 0
for idx in wrong_predictions2:
    if idx not in wrong_predictions1:
        diff.append(idx)
        labels.append(wrong_labels2[i].item())
    i+=1


In [ ]:
len(diff)

In [ ]:
len(labels)

In [ ]:
targeted_examples = [ ]
for i, (data, target) in enumerate(dataloader):
    if i in diff:
        targeted_examples.append(data.data.cpu().squeeze().numpy())

In [ ]:
144/6000*100

In [ ]:
np.round(114/144*100, 2)

In [ ]:
c = 0
for d in diff:
    if d in wrong_predictions3:
        c+=1
c

In [ ]:
wrongpreds_dict = {i:[] for i in range(9) if i!=6}

In [ ]:
for i in range(len(labels)):
    wrongpreds_dict[labels[i]].append(i)

In [ ]:
from random import shuffle

In [ ]:
for key in wrongpreds_dict:
    shuffle(wrongpreds_dict[key])

In [ ]:
fig=plt.figure(figsize=(11, 8))
columns = 8
rows = 5
c = 1
for i in range(1):
    for key in wrongpreds_dict:
        fig.add_subplot(rows, columns, c)
        plt.axis('off')
        plt.title('Predicted: ' + str(key))
        plt.imshow(targeted_examples[wrongpreds_dict[key][i]], cmap='gray')
        c+= 1
for i in range(1, 5):
    for key in wrongpreds_dict:
        fig.add_subplot(rows, columns, c)
        plt.axis('off')
        plt.imshow(targeted_examples[wrongpreds_dict[key][i]], cmap='gray')
        c+= 1
    

plt.tight_layout()
# plt.subplots_adjust(wspace=0.2, hspace=0.2)
plt.savefig('figures/stealthyattacked_samples.png', dpi = 600)
plt.grid(True)
plt.show(block=True)

In [23]:
x1 = 0.7
x2 = 1
x = np.tanh(x1*0.1 + x2)
x

-0.7305938960959438

In [2]:
x = np.zeros(10)
x

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [3]:
x = np.array([0.1, 0.2, 0.25, 0.17, 0.15, 0.18, 0.22, 0.27, 0.3])
y = x+10
y

array([10.1 , 10.2 , 10.25, 10.17, 10.15, 10.18, 10.22, 10.27, 10.3 ])

In [13]:
q1, q3 = np.percentile(x, [25, 75])
iqr = q3-q1
goods = np.where(x>= q1-(-0.4*iqr)) and np.where(x<=q3-0.4*iqr)
x[goods]

array([0.1 , 0.2 , 0.17, 0.15, 0.18])

In [12]:
q1, q3 = np.percentile(y, [25, 75])
iqr = q3-q1
goods = np.where(y>= q1-(-0.4*iqr)) and np.where(y<=q3-0.4*iqr)
y[goods]

array([10.1 , 10.2 , 10.17, 10.15, 10.18])

In [14]:
from sklearn.decomposition import PCA
x = np.random.randn(10, 100000)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)

In [15]:
principalComponents

array([[ -92.32721068,  -10.54440172],
       [-117.62153404, -151.40757556],
       [  25.87606355,  152.31981125],
       [ -58.38243575,  106.3348854 ],
       [  89.38506053,    5.45794789],
       [  46.5262788 ,  -80.0789033 ],
       [-113.31424366,   98.75958211],
       [ 225.57260352,  -32.28715461],
       [ -26.23354029, -150.00791923],
       [  20.51895801,   61.45372777]])

In [43]:
x = np.array([[-0.8933773,   2.34192447],
 [-1.11605931, -0.20962735],
 [-1.1854294,   0.80547189],
 [-0.89899777,  0.04349739],
 [-0.93409179, -1.57158236],
 [-0.77351962, -1.39170668],
 [ 7.76757072,  0.03902375],
 [-1.03105072,  0.5076061 ],
 [-0.93504481, -0.56460721]])
x = abs(x)
y = np.round(x[:, 1]**2, 2)
y

array([5.48, 0.04, 0.65, 0.  , 2.47, 1.94, 0.  , 0.26, 0.32])

In [83]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist


def sklearn_cosine():
    return cosine_similarity(x, y)

def scipy_cosine(x, y):
    return 1. - cdist(x, y, 'cosine')



In [120]:
x = np.array([[1, 2, 8, 4]])
y = np.array([[1, 2, 8, 4]])

In [121]:
cosine_similarity(x, y)

array([[1.]])

In [104]:
x = np.array([0.57319558, 0.64386993, 0.27781297, 0.63473548, 0.65081185, 0.62074379,
  0.70024013, 0.61808296, 0.60591227])


In [107]:
q1, med = np.percentile(x, [25, 50])

In [110]:
(med-q1)

0.014831519999999987

In [112]:
0.63473548 - med

0.013991690000000001

In [113]:
idx1 = [0, 1, 0, 1, 1]
np.where(idx1)

(array([1, 3, 4], dtype=int64),)

In [ ]:
1,6,8
Scores in group  1 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.35531092 1.0504873  1.05674853 0.14186138 0.62329331 0.23437746
 0.1540746  0.27716906 0.97580078 0.28002637]
Scores in group  2 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.42321309 0.79569618 0.7938901  0.17533488 0.43631722 0.29533799
 0.14894544 0.28342619 0.95329274 0.27430859]
Scores in group  3 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.78430178 2.64499022 2.62550979 0.30884367 1.49805682 0.54316944
 0.30856    0.65717002 2.50687971 0.62517459]
Scores in group  4 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.40298295 0.7888872  0.88904877 0.19576626 0.49246147 0.29129507
 0.182227   0.31199999 0.75115886 0.28537315]
Scores in group  5 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [1.41072931 5.87917698 5.97807375 0.64775874 3.43149972 1.17748061
 0.65731229 1.39220285 5.87768144 1.35065412]
Scores in group  6 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.44925811 1.04561053 1.02910046 0.19499363 0.74447355 0.25587848
 0.17715927 0.33206756 0.99373868 0.35072173]
Scores in group  7 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.82747986 3.87332849 3.90153094 0.43207915 2.22039227 0.78046224
 0.44652801 0.91543309 3.86417766 0.88002187]
Scores in group  8 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1] : 5
Distances:  [0.27374691 0.88113548 0.89040093 0.14119082 0.54302721 0.24898939
 0.15085773 0.27792004 0.78709786 0.28610367]
Scores in group  9 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 0] : 4
Distances:  [0.47516384 1.04553699 0.96454943 0.19755708 0.56103079 0.43705538
 0.19539409 0.44586141 0.94805484 0.2853049 ]
Scores in group  10 
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 0] : 4
Distances:  [0.39099546 0.73067387 0.59399344 0.11318163 0.31818258 0.4449765
 0.12042041 0.3726577  0.57888849 0.23482151]
Scores in group  11 
 [0, 1, 1, 0, 1, 0, 0, 0, 1, 0] : 4
Distances:  [0.25693739 0.19037897 0.12994836 0.047577   0.1179866  0.25604501
 0.04853077 0.25364862 0.13811798 0.06394785]

In [288]:
from pyod.models.loci import LOCI
x = np.random.randn(10, 20000)

In [298]:
clf = LOCI(alpha=0.5, k=5)
pred = clf.fit(x)

In [305]:
pred.labels_
import pandas as pd

In [306]:
targets = {'label':pd.Series(['Benign', 'Malignant'])}
colors = ['r', 'g']
for target, color in zip(targets,colors):
    print(target, ': ', color)

label :  r


In [307]:
targets['label'] == 'Benign'

0     True
1    False
dtype: bool

In [76]:
import numpy as np
import hdbscan

In [105]:
x = np.array([ [1.10113724], [1.49476115], [1.16823136], [1.33004554], [1.07602863], [0.7023438],
 [1.00511192], [0.65592329], [0.76731512], [0.75377172] ])
clf = hdbscan.HDBSCAN(min_cluster_size=2, metric='manhattan')
clf.fit(x)
clf.labels_

array([ 0, -1,  0,  0,  0,  1,  0,  1,  1,  1], dtype=int64)

In [75]:
abs(x - med)

array([1.09034019, 7.13774943, 1.08039012, 0.13146367, 2.18236398,
       0.95790255, 0.4678424 , 0.78241814, 1.0227308 , 0.13146367])